# Import corpus

In [1]:
import pandas as pd
import pickle

In [2]:
simplewiki_medcon = pd.read_csv('../datasources/simplewiki/extracted/swiki_sentences_ngrams_filtered.csv', sep=';')
simplewiki_medcon.columns

Index(['source_page', 'sent_id', 'sentence', 'filtered_unigrams',
       'filtered_bigrams', 'filtered_trigrams', 'filtered_quandrigrams',
       'unigrams_grouped', 'bigrams_grouped', 'trigrams_grouped',
       'quandrigrams_grouped'],
      dtype='object')

In [3]:
simplewiki_sentences = simplewiki_medcon.sentence.tolist()
print(len(simplewiki_sentences))
simplewiki_sentences[:10]

12694


['Boil might mean:   Boiling, heating a liquid to the point where it turns into gas   Boil, a type of Staphylococcal infection',
 'Bubonic plague is the best-known form of the disease plague, which is caused by the bacterium Yersinia pestis.',
 'The name bubonic plague is specific for this form of the disease, which enters through the skin, and travels through the lymphatic system.',
 'If the disease is left untreated, it kills about half its victims, in between three and seven days.',
 'The bubonic plague was the disease that caused the Black Death, which killed tens of millions of people in Europe, in the Middle Ages.',
 'Symptoms of this disease include coughing, fever, and black spots on the skin.',
 'There are different kinds of Bubonic plague.',
 'The most common form of the disease is spread by a certain kind of flea, that lives on rats.',
 'Then there is an incubation period which can last from a few hours to about seven days.',
 'Sepsis happens when the bacterium enters the bl

# Load BERT model from bert_embedding

Code from: https://gist.github.com/avidale/c6b19687d333655da483421880441950

In [4]:
import mxnet as mx
from bert_embedding import BertEmbedding
from tqdm import tqdm
import pickle

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
ctx = mx.gpu(0)

# see https://gluon-nlp.mxnet.io/model_zoo/bert/index.html for model and dataset_name
bert = BertEmbedding(ctx=ctx, max_seq_length=256, model='bert_12_768_12', dataset_name='book_corpus_wiki_en_cased', batch_size=8)
# bert = BertEmbedding(ctx=ctx, max_seq_length=256, model='bert_12_768_12', dataset_name='book_corpus_wiki_en_uncased', batch_size=8)


## Create a search index

In [1]:
from sklearn.neighbors import KDTree
import numpy as np


class ContextNeighborStorage:
    def __init__(self, sentences, model):
        self.sentences = sentences
        self.model = model

    def process_sentences(self):
        result = self.model(self.sentences)

        self.sentence_ids = []
        self.token_ids = []
        self.all_tokens = []
        all_embeddings = []
        for i, (toks, embs) in enumerate(tqdm(result)):
            for j, (tok, emb) in enumerate(zip(toks, embs)):
                self.sentence_ids.append(i)
                self.token_ids.append(j)
                self.all_tokens.append(tok)
                all_embeddings.append(emb)
        all_embeddings = np.stack(all_embeddings)
        # we normalize embeddings, so that euclidian distance is equivalent to cosine distance
        self.normed_embeddings = (all_embeddings.T / (all_embeddings**2).sum(axis=1) ** 0.5).T

    def build_search_index(self):
        # this takes some time
        self.indexer = KDTree(self.normed_embeddings)

    def query(self, query_sent, query_word, k=10, filter_same_word=False):
        toks, embs = self.model([query_sent])[0]

        found = False
        for tok, emb in zip(toks, embs):
            if tok == query_word or tok == query_word.lower():
                found = True
                break
        if not found:
            raise ValueError('The query word {} is not a single token in sentence {}'.format(query_word, toks))
        emb = emb / sum(emb**2)**0.5

        if filter_same_word:
            initial_k = max(k, 100)
        else:
            initial_k = k
        di, idx = self.indexer.query(emb.reshape(1, -1), k=initial_k)
        distances = []
        neighbors = []
        contexts = []
        for i, index in enumerate(idx.ravel()):
            token = self.all_tokens[index]
            if filter_same_word and (query_word in token or token in query_word):
                continue
            distances.append(di.ravel()[i])
            neighbors.append(token)
            contexts.append(self.sentences[self.sentence_ids[index]])
            if len(distances) == k:
                break
        return distances, neighbors, contexts

In [9]:
storage = ContextNeighborStorage(sentences=simplewiki_sentences, model=bert)
storage.process_sentences()

100%|██████████| 12694/12694 [00:00<00:00, 160465.63it/s]


In [10]:
storage.build_search_index()

In [11]:
# Store data (serialize)
with open('./simplewiki_KDTree_bertcased256.pickle', 'wb') as handle:
    pickle.dump(storage, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
# Load data (deserialize)
with open('./simplewiki_KDTree_bertcased256.pickle', 'rb') as handle:
    tree_copy = pickle.load(handle)

# Load ClinicalBERT via SBERT

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from transformers import BertForMaskedLM, BertTokenizer
import torch
from tqdm import tqdm
import pickle

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Adjust bert_embedding to load clinicalbert

In [11]:
# Original code from: https://github.com/imgarylai/bert-embedding/blob/master/bert_embedding/bert.py

class BertEmbedding(object):
    def __init__(self, model, max_seq_length=256, do_lower_case=False):
        self.word_embedding_model = models.BERT(model, max_seq_length=max_seq_length, do_lower_case=do_lower_case)
#         self.tokenizer = BertTokenizer.from_pretrained(model, do_lower_case=do_lower_case)
        
    def __call__(self, sentences, oov_way='avg', filter_spec_tokens=True):
        """
        Get tokens, tokens embedding
        Parameters
        ----------
        sentences : List[str]
            sentences for encoding.
        oov_way : str, default avg.
            use **avg**, **sum** or **last** to get token embedding for those out of
            vocabulary words
        filter_spec_tokens : bool
            filter [CLS], [SEP] tokens.
        Returns
        -------
        List[(List[str], List[ndarray])]
            List of tokens, and tokens embedding
        """
        return self.embedding(sentences, oov_way, filter_spec_tokens)

    def embedding(self, sentences, oov_way='avg', filter_spec_tokens=True):
        # Apply mean pooling to get one fixed sized sentence vector
        pooling_model = models.Pooling(self.word_embedding_model.get_word_embedding_dimension(),
                                       pooling_mode_mean_tokens=True,
                                       pooling_mode_cls_token=False,
                                       pooling_mode_max_tokens=False)

        bert = SentenceTransformer(modules=[self.word_embedding_model, pooling_model])

        sentence_embeddings = bert.encode(sentences, output_value='token_embeddings')

        token_ids = []
        for sent in sentences:
            token_ids.append(bert.tokenize(sent))

        batches = []
        for token_id, embedding in zip(token_ids, sentence_embeddings):
            batches.append((token_id, embedding))

        return self.oov(batches, oov_way='avg', filter_spec_tokens=True)

    def oov(self, batches, oov_way='avg', filter_spec_tokens=True):
        """
        How to handle oov. Also filter out [CLS], [SEP] tokens.
        Parameters
        ----------
        batches : List[(tokens_id,
                        sequence_outputs,
                        pooled_output].
            batch   token_ids (max_seq_length, ),
                    sequence_outputs (max_seq_length, dim, ),
                    pooled_output (dim, )
        oov_way : str
            use **avg**, **sum** or **last** to get token embedding for those out of
            vocabulary words
        filter_spec_tokens : bool
            filter [CLS], [SEP] tokens.
        Returns
        -------
        List[(List[str], List[ndarray])]
            List of tokens, and tokens embedding
        """
        sentences = []
        for token_ids, sequence_outputs in batches:
            tokens = []
            tensors = []
            oov_len = 1
            for token_id, sequence_output in zip(token_ids, sequence_outputs):
                if token_id == 1:
                    # [PAD] token, sequence is finished.
                    break
                if (token_id in (2, 3)) and filter_spec_tokens:
                    # [CLS], [SEP]
                    continue
                token = self.word_embedding_model.tokenizer.convert_ids_to_tokens([token_id])[0] #self.tokenizer.convert_ids_to_tokens([token_id])[0] #self.vocab.idx_to_token[token_id]
                if token.startswith('##'):
                    token = token[2:]
                    tokens[-1] += token
                    if oov_way == 'last':
                        tensors[-1] = sequence_output
                    else:
                        tensors[-1] += sequence_output
                    if oov_way == 'avg':
                        oov_len += 1
                else:  # iv, avg last oov
                    if oov_len > 1:
                        tensors[-1] /= oov_len
                        oov_len = 1
                    tokens.append(token)
                    tensors.append(sequence_output)
            if oov_len > 1:  # if the whole sentence is one oov, handle this special case
                tensors[-1] /= oov_len
            sentences.append((tokens, tensors))
        return sentences


## Create a search index

In [12]:
# Original code from: https://gist.github.com/avidale/c6b19687d333655da483421880441950

from sklearn.neighbors import KDTree
import numpy as np


class ContextNeighborStorage:
    def __init__(self, sentences, model):
        self.sentences = sentences
        self.model = model

    def process_sentences(self):
        result = self.model(self.sentences)
        self.sentence_ids = []
        self.token_ids = []
        self.all_tokens = []
        all_embeddings = []
        for i, (toks, embs) in enumerate(tqdm(result)):
            for j, (tok, emb) in enumerate(zip(toks, embs)):
                self.sentence_ids.append(i)
                self.token_ids.append(j)
                self.all_tokens.append(tok)
                all_embeddings.append(emb)
        all_embeddings = np.stack(all_embeddings)
        # we normalize embeddings, so that euclidian distance is equivalent to cosine distance
        self.normed_embeddings = (all_embeddings.T / (all_embeddings**2).sum(axis=1) ** 0.5).T
        print("count self.normed_embeddings: ", len(self.normed_embeddings))

    def build_search_index(self):
        # this takes some time
        self.indexer = KDTree(self.normed_embeddings)

    def query(self, query_sent, query_word, k=10, filter_same_word=False):
        toks, embs = self.model([query_sent])[0]
        found = False
        for tok, emb in zip(toks, embs):
            if tok == query_word:
                found = True
                break
        if not found:
            raise ValueError('The query word "{}" is not a single token in sentence {}'.format(query_word, toks))
        emb = emb / sum(emb**2)**0.5

        if filter_same_word:
            initial_k = max(k, 100)
        else:
            initial_k = k
        di, idx = self.indexer.query(emb.reshape(1, -1), k=initial_k)
        distances = []
        neighbors = []
        contexts = []
        for i, index in enumerate(idx.ravel()):
            token = self.all_tokens[index]
            if filter_same_word and (query_word in token or token in query_word or query_word.lower() in token or token in query_word.lower()):
                continue
            distances.append(di.ravel()[i])
            neighbors.append(token)
            contexts.append(self.sentences[self.sentence_ids[index]])
            if len(distances) == k:
                break
        
        return distances, neighbors, contexts

In [13]:
simplewiki_medcon = pd.read_csv('../datasources/simplewiki/extracted/swiki_sentences_ngrams_filtered.csv', sep=';')
simplewiki_sentences = simplewiki_medcon.sentence.tolist()

In [14]:
path_to_clinicalbert = '../BERT_models/biobert_pretrain_output_disch_100000'
clinicalbert = BertEmbedding(model=path_to_clinicalbert)

INFO:transformers.configuration_utils:loading configuration file ../BERT_models/biobert_pretrain_output_disch_100000/config.json
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

INFO:transformers.modeling_utils:loading weights file ../BERT_models/biobert_pretrain_output_disch_100000/pytorch_model.bin
INFO:transfor

In [15]:
storage = ContextNeighborStorage(sentences=simplewiki_sentences, model=clinicalbert)
storage.process_sentences()

INFO:root:Use pytorch device: cuda
100%|██████████| 12694/12694 [00:00<00:00, 179225.42it/s]


count self.normed_embeddings:  238413


In [7]:
storage.build_search_index()

In [8]:
# Store data (serialize)
with open('./simplewiki_KDTree_clinicalbert.pickle', 'wb') as handle:
    pickle.dump(storage, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
# Load data (deserialize)
with open('./simplewiki_KDTree_clinicalbert.pickle', 'rb') as handle:
    tree_copy = pickle.load(handle)

In [27]:
# tree_copy.indexer.get_arrays()

In [28]:
# tree_copy.indexer.get_tree_stats()

# Experiments

__Meningitis__ \
CHV_selection: ['meningitis', 'meningitis/encephalitis', 'pachymeningitis'] \
"Alexandra died in 1869 from meningitis." \
"It would if she developed meningitis." \
"This man has meningitis." \
"He doesn't even have meningitis." \
"There's no evidence of meningitis on that MRI." \
"Well, then she's dying, 'cause the meningitis treatment isn't helping her." \
"How about them cheap meningitis drugs they pawning off in Africa?" \
"No meningitis, no other infections." \
"She could be, but a meningitis patient will be without a CT scan." \
"The antibiotic we gave you in case you had meningitis can turn your tears reddish." \
"If it's meningitis, we have to lD the bug fast." \
"If it was meningitis, we'd all be sick." \
"Progression's too fast.More likely meningitis." \
"LP was negative for meningitis." \
"Just like everybody else we've had to give meningitis shots to." \
"They swap spit, virus travels to his brain, leads to meningitis, leads to increased intracranial pressure." \
"What if it was caused by nf2 cancer instead of his childhood meningitis?" \
"Run CSF tests, find out what's causing the meningitis."

__Gingivitis__ \
CHV_selection: ['gum inflammation'] \
"The leaves and bark are used for controlling blood pressure and gingivitis."

__Atrophy__ \
CHV_selection: ['atrophy', 'atrophic'] \
"Unable to function, the muscles weaken and exhibit atrophy." \
"If you don't exercise it, it'll atrophy, just like anything else."

__SMA__ \
CHA_selection: ['spinal muscular atrophy', 'kennedy syndrome'] \
"Infection stresses his already weakened system, makes his SMA worse."

__leukoencephalopathy__ \
CHV_selection: ['leukoencephalopathy'] \
"What about leukoencephalopathy in a 16-year-old?"

__lameness__ \
CHV_selection: ['claudication'] \
"One of his arms and both feet became enlarged and at some point during his childhood he fell and damaged his hip, resulting in permanent lameness."

__Myelofibrosis__ \
CHV_selection: ['agnogenic myeloid metaplasia', 'idiopathic myelofibrosis', 'myelosclerosis with myeloid metaplasia', 'primary myelofibrosis', 'myelofibrosis'] \
CHV_sub: "idiopathic myelofibrosis" \
"Myelofibrosis fits, but testing takes at least 48 hours."

__IBD__ \
CHV_selection: ['inflammatory bowel disease', 'irritable bowel syndrome (IBS)'] \
"Biopsy showed non-specific inflammation, which suggests IBD."

__hypoperfusion__ \
CHV_selection: ['hypoperfusion', 'respiratory distress syndrome'] \
"But SARS explains the cough, causes hypoperfusion, which explains the ischemic bowel." \
"There's a slight hypoperfusion in the anterior cortex." \
"The condition usually results from infection, injury (accident, surgery), hypoperfusion and hypermetabolism."

__melanoma__ \
CHV_selection: ['melanoma'] \
"Check for melanomas." \
"From considering melanoma as a diagnosis." \
"Unfortunately, it also prevents even those with advanced medical degrees from considering melanoma as a diagnosis." \
"The biopsy revealed it's melanoma." \
"The woman has melanoma." \
"The woman has melanoma, she could die if we don't..." \
"Since the baby has the melanoma, we kind of already knew the mom had it." \
"What's interesting is that mom's melanoma spread to the baby in utero, which means it's metastatic, which means it's late stage." \
"How many people with a late stage melanoma are as healthy as she is?" \
"Clearly, something's going on in mom's blood that's treating her melanoma as well as her baby's." \
"If her immune system was fighting the melanoma, it never would have spread." \
"Exactly, which means it's fighting something else, and the melanoma cells are getting caught in the crossfire." \
"Could his dad's melanoma be a factor here?" \
"If the kid's dad died young of melanoma, how come the overprotective mom never took him to a dermatologist?" \
"His family history of melanoma may have affected his cellular outlook." \
"If we had more data on the exact type of melanoma his father had..."

__Granuloma__ \
CHV_selection: ['granuloma', 'granulomatous lesion'] \
CHV_sub: "granulomatous lesion" \
"Or infections that cause granulomas," \
"Time to scan her for infections that could cause granulomas." \
"Lungs clear for granulomas." \
'The nodules in his spleen were granulomas.' \
"He could have had a granuloma in his sinuses that bled, which could have been caused by Wegener's." \
"Could be granulomas, could be plaques." \
"We can nuke them with a mild course of chemo and then remove the granuloma." \
"Looked like a granuloma on the MRI, but it's not." \
"He's got wegener's, which means he's got granulomas." \
"MRI showed a granuloma in his liver." \
"He had a granuloma in his liver." \
"They found granulomas in Lucy's pericardium."

__sarcoidosis__ \
CHV_selection: ['sarcoidosis'] \
"But the calcified pineal could be sarcoidosis." \
"The PET scan was negative for sarcoidosis." \
"Sarcoidosis is a diagnosis of exclusion, which means that the cowards who spend their time excluding what it isn't usually cause their patients to die from what it is." \
"Sarcoidosis could be inflamed by the treatment." \
"I'm fairly confident our patient's got sarcoidosis." \
"We ran some tests and the results point toward sarcoidosis." \
"And triggered a dormant sarcoidosis." \
"Sarcoidosis could be in his brain and lungs." \
"If sarcoidosis has two environmental causes, it has environmental causes." \
"Sarcoidosis isn't infectious or environmental." \
"Sarcoidosis explains almost all of their symptoms." \
"A.C.E. Levels are too low for sarcoidosis." \
"Maggie tested negative for sarcoidosis." \
"Sarcoidosis hits the spleen and the liver." \
"Sarcoidosis is progressive." \
"The heart could point to sarcoidosis."

__hemosiderosis__ \
CHV_selection: ['hemosiderosis'] \
"The donor had hemosiderosis."

__amyloidosis__ \
CHV_selection: ['amyloidosis'] \
"The largest risk factors for spontaneous bleeding are high blood pressure and amyloidosis." \
"Negative for amyloidosis." \
"Plus internal bleeding, equals amyloidosis." \
"He's bleeding way too much for amyloidosis." \
"Except that nothing in his medical history remotely indicates amyloidosis." \
"He needs a bone marrow transplant for the amyloidosis." \
"Only way to treat amyloidosis is to treat whatever's causing it." \
"Amyloidosis with systemic involvement." \
"If you had amyloidosis, there'd be speckling on your cardiac ultrasound." \
"We'll do a kidney biopsy to test for amyloidosis." \
"Biopsy showed deposits in the renal endothelium consistent with amyloidosis."

__laryngospasm__ \
CHV_selection: ['laryngospasm'] \
"Do a bronchoscopy, it'll set off a laryngospasm." \
"This is our chance to prove laryngospasm."

__porphyria__ \
CHV_selection: ['porphyria'] \
"If she had porphyria, you would have seen purple urine." \
"Pulmonary involvement rules out porphyria." \
"It's also a porphyria trigger." \
"I was wrong about the porphyria, but I wasn't wrong about the nervous system." \
"If you're wrong about porphyria, the treatment could box her kidneys." \
"it's porphyria and it's moving fast."

__nystagmus__ \
CHV_selection: ['nystagmus'] \
"Also, many blind people have nystagmus, which is one reason that some wear dark glasses." \
"There are two key forms of nystagmus: pathological and physiological, with variations within each type."

__bronchiectasis__ \
CHV_selection: ['bronchiectasis'] \
"People with bronchiectasis may have bad breath indicative of active infection." \
"Some people with bronchiectasis may produce frequent green/yellow sputum (up to 240ml (8 oz) daily)." \
"Bronchiectasis has both congenital and acquired causes, with the latter more frequent."

In [3]:
sentences = [
        ("The antibiotic we gave you in case you had meningitis can turn your tears reddish.",
         "meningitis"),
        ("They swap spit, virus travels to his brain, leads to meningitis, leads to increased intracranial pressure.",
         "meningitis"),
        ("The leaves and bark are used for controlling blood pressure and gingivitis.",
         "gingivitis"),
        ("Unable to function, the muscles weaken and exhibit atrophy.", "atrophy"),
        ("If you don't exercise it, it'll atrophy, just like anything else.",
         "atrophy"),
        ("Infection stresses his already weakened system, makes his SMA worse.",
         "SMA"),
        ("One of his arms and both feet became enlarged and at some point during his childhood he fell and damaged his hip, resulting in permanent lameness.",
         "lameness"),
        ("But SARS explains the cough, causes hypoperfusion, which explains the ischemic bowel.",
         "hypoperfusion"),
        ("There's a slight hypoperfusion in the anterior cortex.",
         "hypoperfusion"),
        ("The condition usually results from infection, injury (accident, surgery), hypoperfusion and hypermetabolism.",
         "hypoperfusion"),
        ("Myelofibrosis fits, but testing takes at least 48 hours.",
         "Myelofibrosis"),
        ("Biopsy showed non-specific inflammation, which suggests IBD.", "IBD"),
        ("The biopsy revealed it's melanoma.", "melanoma"),
        ("What's interesting is that mom's melanoma spread to the baby in utero, which means it's metastatic, which means it's late stage.",
         "melanoma"),
        ("How many people with a late stage melanoma are as healthy as she is?",
         "melanoma"),
        ("His family history of melanoma may have affected his cellular outlook.",
         "melanoma"),
        ("If the kid's dad died young of melanoma, how come the overprotective mom never took him to a dermatologist?",
         "melanoma"),
        ("He could have had a granuloma in his sinuses that bled, which could have been caused by Wegener's.",
         "granuloma"), ("MRI showed a granuloma in his liver.", "granuloma"),
        ("We can nuke them with a mild course of chemo and then remove the granuloma.",
         "granuloma"),
        ("Time to scan her for infections that could cause granulomas.",
         "granulomas"),
        ("But the calcified pineal could be sarcoidosis.", "sarcoidosis"),
        ("Sarcoidosis could be in his brain and lungs.", "Sarcoidosis"),
        ("The PET scan was negative for sarcoidosis.", "sarcoidosis"),
        ("Sarcoidosis isn't infectious or environmental.", "Sarcoidosis"),
        ("The donor had hemosiderosis.", "hemosiderosis"),
        ("The largest risk factors for spontaneous bleeding are high blood pressure and amyloidosis.",
         "amyloidosis"),
        ("If you had amyloidosis, there'd be speckling on your cardiac ultrasound.",
         "amyloidosis"),
        ("Biopsy showed deposits in the renal endothelium consistent with amyloidosis.",
         "amyloidosis"),
        ("He needs a bone marrow transplant for the amyloidosis.", "amyloidosis"),
        ("Do a bronchoscopy, it'll set off a laryngospasm.", "laryngospasm"),
        ("If she had porphyria, you would have seen purple urine.", "porphyria"),
        ("If you're wrong about porphyria, the treatment could box her kidneys.",
         "porphyria"),
        ("Also, many blind people have nystagmus, which is one reason that some wear dark glasses.",
         "nystagmus"),
        ("There are two key forms of nystagmus: pathological and physiological, with variations within each type.",
         "nystagmus"),
        ("People with bronchiectasis may have bad breath indicative of active infection.",
         "bronchiectasis"),
        ("Some people with bronchiectasis may produce frequent green/yellow sputum (up to 240ml (8 oz) daily).",
         "bronchiectasis"),
        ("Bronchiectasis has both congenital and acquired causes, with the latter more frequent.",
         "Bronchiectasis"),
        ("However, Satie's military career did not last very long; within a few months he was discharged after deliberately infecting himself with bronchitis.",
         "bronchitis"),
        ("In January 1876, his father died of bronchitis following a long period of depression.",
         "bronchitis"),
        ("She died in The Hague on 20 March 1934, of complications from bronchitis at the age of 75, and was buried in Delft.",
         "bronchitis"),
        ("She died in The Hague on 20 March 1934, of complications from bronchitis at the age of 75.",
         "bronchitis"),
        ("Menuhin died in Martin Luther Hospital, Berlin, Germany, from complications of bronchitis.",
         "bronchitis"),
        ("It is one of the rarest forms of synesthesia.", "synesthesia"),
        ("An outbreak of tularemia occurred in Kosovo in 1999-2000.", "tularemia"),
        ("Over the following summers, Martha's Vineyard was identified as the only place in the world where documented cases of tularemia resulted from lawn mowing.",
         "tularemia"),
        ("Tularemia is not spread directly from person to person.", "Tularemia"),
        ("There is currently no effective treatment or cure for akinetopsia.",
         "akinetopsia"),
        ("Patients with akinetopsia struggle with many issues in their day-to-day life, depending on the severity of their condition.",
         "akinetopsia"),
        ("Besides simple perception, akinetopsia also disturbs visuomotor tasks, such as reaching for objects and catching objects.",
         "akinetopsia"),
        ("Inconspicuous akinetopsia can be selectively and temporarily induced using transcranial magnetic stimulation (TMS) of area V5 of the visual cortex in healthy subjects.",
         "akinetopsia"),
        ("Only in a few cases and after many years does it cause demyelination.",
         "demyelination"),
    ("Ho was diagnosed with cardiomyopathy in 2005 and had a pacemaker implanted.",
     "cardiomyopathy"),
    ("He was diagnosed with cardiomyopathy in 2005 and had a pacemaker implanted.",
     "cardiomyopathy"),
    ("His echocardiogram was negative for cardiomyopathy.", "cardiomyopathy"),
    ("And a subsequent biopsy revealed irreversible cardiomyopathy.",
     "cardiomyopathy"),
    ("The kid has severe cardiomyopathy.", "cardiomyopathy"),
    ("Which means it could be a hematological problem plus cardiomyopathy",
     "cardiomyopathy"),
    ("An acute myocardial infarction, happens when a blood vessel in the heart suddenly becomes blocked.",
     "myocardial"),
    ("Patau syndrome is a syndrome caused by a chromosomal abnormality, in which some or all of the cells of the body contain extra genetic material from chromosome 13.",
     "Patau"),
    ("Most cases of Patau syndrome are not inherited, but occur as random events during the formation of reproductive cells (eggs and sperm).",
     "Patau"),
    ("But a baby with Patau syndrome has 3 copies of chromosome 13, instead of 2.",
     "Patau"),
    ("Nora is diagnosed with Trisomy 13.", "Trisomy"),
    ("The patient has trisomy 13.", "trisomy"),
    ("The patient has trisomy 21.", "trisomy"),
    ("Rarely very low blood pressure may be the only sign of anaphylaxis.",
     "anaphylaxis"),
    ("Worldwide, 0.05-2% of the population is estimated to have anaphylaxis at some point in life, and rates appear to be increasing.",
     "anaphylaxis"),
    ("On a mechanistic level, anaphylaxis is caused by the release of mediators from certain types of white blood cells triggered either by immunologic or non-immunologic mechanisms.",
     "anaphylaxis"),
    ("Previous systemic reactions, which are anything more than a local reaction around the site of the sting, are a risk factor for future anaphylaxis; however, half of fatalities have had no previous systemic reaction.",
     "anaphylaxis"),
    ('In a person who died from anaphylaxis, autopsy may show an "empty heart" attributed to reduced venous return from vasodilation and redistribution of intravascular volume from the central to the peripheral compartment.',
     "anaphylaxis"),
    ("Anaphylaxis can occur in response to almost any foreign substance.",
     "Anaphylaxis"),
    ("Many foods can trigger anaphylaxis; this may occur upon the first known ingestion.",
     "anaphylaxis"),
    ('People prone to anaphylaxis are advised to have an "allergy action plan."',
     "anaphylaxis"),
    ("The antitoxin could cause anaphylaxis.", "anaphylaxis"),
    ("A food allergy explains the anaphylaxis.", "anaphylaxis"),
    ("They die of dysentery.", "dysentery"),
    ("Henry II moved in support of Richard, and Henry the Young King died from dysentery at the end of the campaign.",
     "dysentery"),
    ("Once at Richmond, Madison began drafting the Report, though he was delayed by a weeklong battle with dysentery.",
     "dysentery"),
    ("Psoriasis varies in severity from small, localized patches to complete body coverage.",
     "Psoriasis"),
    ("Psoriasis is generally thought to be a genetic disease that is triggered by environmental factors.",
     "Psoriasis"),
    ("There are five main types of psoriasis: plaque, guttate, inverse, pustular, and erythrodermic.",
     "psoriasis"),
    ("If one twin has psoriasis, the other twin is three times more likely to be affected if the twins are identical than if they are non-identical.",
     "psoriasis"),
    ("High doses may lead to muscles contractions.", "contractions"),
    ("High doses may lead to convulsions.", "convulsions"),
    ("To reduce the encephalocele cyst.", "encephalocele"),
    ("Epistaxis in children is usually from Little's area, which is on the septal wall anteriorly.",
     "Epistaxis"),
    ("Once epistaxis occurs, the importance of the first treatment for the haemostasis should be emphasized.",
     "epistaxis")
]

In [4]:
import sys
sys.path.insert(1, '../LexSimp_BERT_MLM/')

from transformers import BertForMaskedLM, BertTokenizer
import torch
import MLM_LS
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity as cosine
import pandas as pd
from tqdm import tqdm
import pickle

In [5]:
def substitution_selection(source_word, closest_candidates, stopwords, ps, num_selection):
    cur_tokens = []
    cur_contexts = []

    source_stem = ps.stem(source_word)

    assert num_selection <= len(closest_candidates)

    punctuationMarks = [',', '.', ';', ':', '(', '-', ')', '{', '}', '[', ']', '\\', '/', '!', '?', '<', '>', '"', "'"]
    
#     candidates = []
#     contexts = []
    for word, context in closest_candidates:
#         candidates.append(word)
#         contexts.append(context)
    
#     for i in range(len(candidates)):
#         token = candidates[i]
        token = word

        if token[0:2] == "##":
            continue

        if (token == source_word):
            continue

        if str(token).isdigit():
            continue

        if token.isalpha() and len(token) == 1:
            continue

        if token in punctuationMarks:
            continue

        if token.lower() in stopwords:
            continue

        token_stem = ps.stem(token)

        if (token_stem.lower() == source_stem.lower()):
            continue

        # if (len(token_stem) >= 3) and (token_stem[:3].lower() == source_stem[:3].lower()):
        #     continue  
        
        if token.lower() in [x.lower() for x in cur_tokens]:
            continue
            
        cur_tokens.append(token)
        
        cur_contexts.append(context)

        if (len(cur_tokens) == num_selection):
            break

    if (len(cur_tokens) == 0):
        candidates = closest_candidates[0:num_selection + 1]
        for d, w, c in candidates:
            cur_tokens.append(w)
            cur_contexts.append(c)
            
    assert len(cur_tokens) > 0
    assert len(cur_tokens) == len(cur_contexts)
    
    ss = list(zip(cur_tokens, cur_contexts))

    return ss

In [6]:
def get_score(sentence, tokenizer, maskedLM):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    tensor_input = tensor_input.to('cuda')
    sentence_loss = 0

    for i, word in enumerate(tokenize_input):
        original_word = tokenize_input[i]
        tokenize_input[i] = '[MASK]'
        # print(tokenize_input)
        mask_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
        mask_input = mask_input.to('cuda')
        with torch.no_grad():
            word_loss = maskedLM(mask_input, masked_lm_labels=tensor_input)[0].data.cpu().numpy()
        sentence_loss += word_loss
        tokenize_input[i] = original_word

    return np.exp(sentence_loss / len(tokenize_input))


def LM_score(source_word, source_context, substitution_selection, tokenizer, maskedLM):
    # source_index = source_context.index(source_word)

    source_sentence = ''

    for context in source_context:
        source_sentence += context + " "

    source_sentence = source_sentence.strip()
    # print(source_sentence)
    LM = []

    for substibution, context in substitution_selection:
        sub_sentence = source_sentence.replace(source_word, substibution)

        # print(sub_sentence)
        score = get_score(sub_sentence, tokenizer, maskedLM)
        LM.append(score)

    return LM

def preprocess_SR(source_word, substitution_selection, fasttext_dico, fasttext_emb, word_count):
    ss = []
    # ss_score=[]
    sis_scores=[]
    count_scores=[]
    # source_count = 10
    # if source_word in word_count:
    #     source_count = word_count[source_word]

    isFast = True

    if (source_word not in fasttext_dico):
        isFast = False
    else:
        source_emb = fasttext_emb[fasttext_dico.index(source_word)].reshape(1,-1)

    if isFast == False and source_word.lower() in fasttext_dico:
        isFast = True
        source_emb = fasttext_emb[fasttext_dico.index(source_word.lower())].reshape(1,-1)

    # ss.append(source_word)

    for sub, context in substitution_selection:

        if sub.lower() not in word_count:
            continue
        else:
            sub_count = word_count[sub.lower()]

        # if sub_count<source_count:
        #     continue
        if isFast:
            if sub not in fasttext_dico:
                if sub.lower() not in fasttext_dico:
                    continue
                else:
                    sub_emb = fasttext_emb[fasttext_dico.index(sub.lower())].reshape(1, -1)
            else:
                sub_emb = fasttext_emb[fasttext_dico.index(sub)].reshape(1, -1)

            sis = cosine(source_emb, sub_emb)[0][0]

            # if sis<0.35:
            #    continue
            sis_scores.append(sis)

        ss.append((sub, context.strip()))
        count_scores.append(sub_count)

    return ss, sis_scores, count_scores

def substitution_ranking(source_word, source_context, substitution_selection, fasttext_dico, fasttext_emb, word_count,
                         tokenizer, maskedLM):
    ss, sis_scores, count_scores = preprocess_SR(source_word, substitution_selection, fasttext_dico, fasttext_emb,
                                                 word_count)

    if len(ss) == 0:
        return source_word

    if len(sis_scores) > 0:
        seq = sorted(sis_scores, reverse=True)
        sis_rank = [seq.index(v) + 1 for v in sis_scores]

    rank_count = sorted(count_scores, reverse=True)
    count_rank = [rank_count.index(v) + 1 for v in count_scores]

    lm_score = LM_score(source_word, source_context, ss, tokenizer, maskedLM)
    rank_lm = sorted(lm_score)
    lm_rank = [rank_lm.index(v) + 1 for v in lm_score]

    bert_rank = []
    for i in range(len(ss)):
        bert_rank.append(i + 1)

    if len(sis_scores) > 0:
        all_ranks = [bert + sis + count + LM for bert, sis, count, LM in zip(bert_rank, sis_rank, count_rank, lm_rank)]
    else:
        all_ranks = [bert + count + LM for bert, count, LM in zip(bert_rank, count_rank, lm_rank)]

#     print("bert_rank: ", bert_rank)
#     if len(sis_scores) > 0:
#         print("sis_rank: ", sis_rank)
#     print("count_rank: ", count_rank)
#     print("lm_rank: ", lm_rank)
#     print("all_ranks: ", all_ranks)
    
    substitution_rank = list(zip(ss, all_ranks))
    sort_substitution_rank = sorted(substitution_rank, key=lambda x: x[1])
#     print("Sorted substitution rank:\n", sort_substitution_rank)
    
    sorted_substitutions = []
    for i, item in enumerate(sort_substitution_rank):
        sorted_substitutions.append(item[0])

    return sorted_substitutions


In [7]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('../BERT_models/biobert_pretrain_output_disch_100000', do_lower_case=False)

INFO:transformers.tokenization_utils:Model name '../BERT_models/biobert_pretrain_output_disch_100000' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1). Assuming '../BERT_models/biobert_pretrain_output_disch_100000' is a path or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:Didn't find file ../BERT_models/biobert_pretrain_output_disch_100000/added_tokens.json. We won't load it.
INFO:transformers.tokenization_utils:Didn't find file ../BERT_models/biobert_

In [8]:
# Load fastText word embeddings
print("Loading embeddings ...")
wordVecPath = '../lex_simp_with_masked_language_model/word_embeddings/fastText/crawl-300d-2M-subword.vec'
fasttext_dico, fasttext_emb = MLM_LS.getWordmap(wordVecPath)

# Load word frequency
word_count_path = '../lex_simp_with_masked_language_model/word_frequency/counter_Tokens.p'
with open(word_count_path, 'rb') as f:
    word_count = pickle.load(f)

# Load BERT MLM
model = BertForMaskedLM.from_pretrained('../BERT_models/biobert_pretrain_output_disch_100000') #, do_lower_case=False)
model.to('cuda')
model.eval()

Loading embeddings ...
2000000 300



INFO:transformers.configuration_utils:loading configuration file ../BERT_models/biobert_pretrain_output_disch_100000/config.json
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

INFO:transformers.modeling_utils:loading weights file ../BERT_models/biobert_pretrain_output_disch_100000/pytorch_model.bin
INFO:transfor

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [10]:
# Load stop words
stopword_list1 = set(stopwords.words('english'))
with open('../lex_simp_with_masked_language_model/MySQL_MyISAM_stopwords.txt', "r") as f:
    stopword_list2 = set(eval(f.read()))
stopword_list = stopword_list1.union(stopword_list2)

ps = PorterStemmer()

num_selection = 12

## ClinicalBERT

### Selection > ranking

In [23]:
def output_ss_sr(query_sent, query_word, k):

    tokens, words, position = MLM_LS.convert_sentence_to_token(sentence=query_sent, tokenizer=tokenizer, seq_length=256)

#     print("tokens: ", tokens)
#     print("words: ", words)
#     print("position: ", position)

    distances, neighbors, contexts= tree_copy.query(query_sent=query_sent, query_word=query_word, k=k, filter_same_word=True)

    closest_candidates = list(zip(neighbors, contexts))
#     print(" ")
#     print(len(closest_candidates))
#     print(closest_candidates)
#     print(" ")

    ss = substitution_selection(source_word=query_word,
                                closest_candidates=closest_candidates,
                                stopwords=stopword_list,
                                ps=ps,
                                num_selection=num_selection)
#     print(len(ss))
#     print(ss)

    mask_index = words.index(query_word)
#     print("mask_index: ", mask_index)
    window_context = 11
    mask_context = MLM_LS.extract_context(words, mask_index, window_context)

    sr = substitution_ranking(source_word=query_word,
                            source_context=mask_context,
                            substitution_selection=ss,
                            fasttext_dico=fasttext_dico,
                            fasttext_emb=fasttext_emb,
                            word_count=word_count,
                            tokenizer=tokenizer,
                            maskedLM=model)
    
    print(" ")
    print("query_sent: ", query_sent)
    print("query_word: ", query_word)
    print("Top substitutions:")
    for i, item in enumerate(sr):
        print(f'{i+1}. {item}')


In [24]:
for query_sent, query_word in tqdm(sentences):
    output_ss_sr(query_sent, query_word, k=50)

  0%|          | 0/34 [00:00<?, ?it/s]INFO:root:Use pytorch device: cuda

  3%|▎         | 1/34 [00:02<01:07,  2.04s/it]INFO:root:Use pytorch device: cuda

Batches: 100%|██████████| 1/1 [00:00<00:00, 104.16it/s]

 
query_sent:  Ho was diagnosed with cardiomyopathy in 2005 and had a pacemaker implanted.
query_word:  cardiomyopathy
Top substitutions:
1. ('arrhythmia', 'Heart   Heart disease   Cardiac arrhythmia')
2. ('rhabdomyosarcoma', 'For example, adults with rhabdomyosarcoma usually have tumors that grow faster and are harder to treat.')
3. ('Myocarditis', 'Myocarditis, also known as inflammatory cardiomyopathy, is inflammation and infection of the heart muscle.')
4. ('cardiogenic', 'It can also be caused by cardiogenic shock, where the heart is not getting enough blood to pump out to the rest of the body.')
5. ('myocardial', 'redirect myocardial infarction')
6. ('leiomyosarcomas', 'It has been believed that leiomyosarcomas do not arise from leiomyomas.')
7. ('myotonic', 'There is no cure for myotonic dystrophy.')
8. ('ciliopathy', 'It is a ciliopathy.')
9. ('transplantation', 'Patients with this disease usually undergo heart transplantation.')
10. ('dysthymia', 'People with dysthymia have de


  6%|▌         | 2/34 [00:03<01:02,  1.94s/it]INFO:root:Use pytorch device: cuda

Batches: 100%|██████████| 1/1 [00:00<00:00, 103.28it/s]

 
query_sent:  His echocardiogram was negative for cardiomyopathy.
query_word:  cardiomyopathy
Top substitutions:
1. ('cardiogenic', 'It can also be caused by cardiogenic shock, where the heart is not getting enough blood to pump out to the rest of the body.')
2. ('pathophysiology', 'This change has occurred because the older dichotomous classification did not reflect pathophysiology or outcome.')
3. ('tachycardia', 'A doctor can diagnose ventricular tachycardia by doing an electrocardiogram (also called an ECG or an EKG).')
4. ('myocardial', 'redirect myocardial infarction')
5. ('dystrophy', 'There is no cure for myotonic dystrophy.')
6. ('myotonic', 'There is no cure for myotonic dystrophy.')
7. ('Myocarditis', 'Myocarditis, also known as inflammatory cardiomyopathy, is inflammation and infection of the heart muscle.')
8. ('leiomyosarcomas', 'It has been believed that leiomyosarcomas do not arise from leiomyomas.')
9. ('cardioversion', 'If a person still has a pulse, ventricular tach


  9%|▉         | 3/34 [00:05<00:59,  1.93s/it]INFO:root:Use pytorch device: cuda

Batches: 100%|██████████| 1/1 [00:00<00:00, 104.06it/s]

...

 
query_sent:  To reduce the encephalocele cyst.
query_word:  encephalocele
Top substitutions:
1. ('encephalitis', 'This is called " secondary encephalitis. "')
2. ('Encephalopathy', 'Encephalopathy means problem of the brain.')
3. ('encephalomyelitis', 'Neurological disorders: These are problems with the brain and the nerves, like Guillain–Barré syndrome and Post-dengue acute disseminated encephalomyelitis.')
4. ('ciliopathy', 'It is a ciliopathy.')
5. ('cephalic', 'It is a type of cephalic disorder.')
6. ('cranium', 'Also, mountain climbers at high altitude have been known to experience cerebral oedema primarily due to reduced air pressure acting on the cranium.')
7. ('ciliopathies', 'The underlying cause of the ciliopathies may be defects in the mechanism of cilia in the cell.')
8. ('tonsillitis', 'There are treatments for tonsillitis.')
9. ('cremasteric', 'Stress can also make the cremasteric reflex work.')



 97%|█████████▋| 33/34 [00:57<00:01,  1.72s/it]INFO:root:Use pytorch device: cuda

Batches: 100%|██████████| 1/1 [00:00<00:00, 101.41it/s]

 
query_sent:  Epistaxis in children is usually from Little's area, which is on the septal wall anteriorly.
query_word:  Epistaxis
Top substitutions:
1. ('Anaphylaxis', 'Anaphylaxis has many symptoms, such as an itchy rash, throat swelling, breathing problems, and low blood pressure.')
2. ('Anosmia', 'Anosmia is losing the the sense of smell.')
3. ('Prognathism', 'Prognathism: part of the face sticks out more than normal usually the lower jaw.')
4. ('Epidemic', 'Epidemic typhus')
5. ('Angioedema', 'Angioedema may be caused by an allergic reaction.')
6. ('Agoraphobia', 'Agoraphobia is a medical condition that causes anxiety.')
7. ('Anthrax', 'Anthrax, or splenic fever, is a disease.')
8. ('Catalepsy', 'Catalepsy is a nervous condition.')
9. ('Trichomoniasis', "Trichomoniasis is only spread through sex, or if two people's genitals touch.")
10. ('Scaphocephaly', 'Scaphocephaly is a medical disorder.')
11. ('Arachnophobia', 'Arachnophobia is a fear of spiders.')



100%|██████████| 34/34 [00:59<00:00,  1.74s/it]

 
query_sent:  Once epistaxis occurs, the importance of the first treatment for the haemostasis should be emphasized.
query_word:  epistaxis
Top substitutions:
1. ('anaphylaxis', 'Epinephrine is the best and first treatment used for anaphylaxis.')
2. ('angioedema', 'It is possible to have angioedema without hives.')
3. ('anosmia', 'In some cases, accupuncture was successfully used to treat anosmia.')
4. ('episodic', 'Some people who start off getting episodic migraines may start to get chronic migraines later.')
5. ('ectoparasite', 'The mite is an ectoparasite.')
6. ('phobias', 'Causes and risk factors phobias is very.')
7. ('Psittacosis', 'In rare cases, Psittacosis can cause endocarditis - swelling of joints, and swelling of the cornea.')
8. ('trichomoniasis', 'Treatment for trichomoniasis is usually metronidazole (Flagyl), which kills protozoa like Trichomonas vaginalis.')
9. ('anaphylactic', 'They usually have fewer anaphylactic episodes, which are less severe, as they get older.')

### Ranking > selection

In [11]:
def output_sr_ss(query_sent, query_word, k):

    tokens, words, position = MLM_LS.convert_sentence_to_token(sentence=query_sent, tokenizer=tokenizer, seq_length=256)

    #print("tokens: ", tokens)
    #print("words: ", words)
    #print("position: ", position)

    distances, neighbors, contexts= tree_copy.query(query_sent=query_sent, query_word=query_word, k=k, filter_same_word=True)

    closest_candidates = list(zip(neighbors, contexts))
    #print(" ")
    #print(len(closest_candidates))
    #print(closest_candidates)
    #print(" ")

    mask_index = words.index(query_word)
    #print("mask_index: ", mask_index)
    window_context = 11
    mask_context = MLM_LS.extract_context(words, mask_index, window_context)

    sr = substitution_ranking(source_word=query_word,
                            source_context=mask_context,
                            substitution_selection=closest_candidates,
                            fasttext_dico=fasttext_dico,
                            fasttext_emb=fasttext_emb,
                            word_count=word_count,
                            tokenizer=tokenizer,
                            maskedLM=model)

    ss = substitution_selection(source_word=query_word,
                                closest_candidates=sr,
                                stopwords=stopword_list,
                                ps=ps,
                                num_selection=num_selection)
    #print(len(ss))
    #print(ss)
    
    print(" ")
    print("query_sent: ", query_sent)
    print("query_word: ", query_word)
    print("Top substitutions:")
    for i, item in enumerate(ss):
        print(f'{i+1}. {item}')


In [19]:
for query_sent, query_word in tqdm(sentences):
    output_sr_ss(query_sent, query_word, k=50)


  0%|          | 0/29 [00:00<?, ?it/s]INFO:root:Use pytorch device: cuda


Batches: 100%|██████████| 1/1 [00:00<00:00, 54.55it/s]

  3%|▎         | 1/29 [00:07<03:35,  7.71s/it]INFO:root:Use pytorch device: cuda


Batches: 100%|██████████| 1/1 [00:00<00:00, 103.73it/s]

 
query_sent:  Ho was diagnosed with cardiomyopathy in 2005 and had a pacemaker implanted.
query_word:  cardiomyopathy
Top substitutions:
1. ('arrhythmia', 'Heart   Heart disease   Cardiac arrhythmia')
2. ('tachycardia', 'Some people with ventricular tachycardia do not have any symptoms, especially if the V-tach lasts for only a few seconds.')
3. ('ventricular', 'This is called idiopathic ventricular tachycardia.')
4. ('Myocarditis', 'Myocarditis, also known as inflammatory cardiomyopathy, is inflammation and infection of the heart muscle.')
5. ('rhabdomyosarcoma', 'For example, adults with rhabdomyosarcoma usually have tumors that grow faster and are harder to treat.')
6. ('dystrophy', 'There is no cure for myotonic dystrophy.')
7. ('myocardial', 'redirect myocardial infarction')
8. ('bradycardia', 'Also, if bradycardia gets very bad, the heart will be unable to pump enough blood and oxygen to the body.')
9. ('cardiogenic', 'It can also be caused by cardiogenic shock, where the heart 



  7%|▋         | 2/29 [00:14<03:20,  7.43s/it]INFO:root:Use pytorch device: cuda


Batches: 100%|██████████| 1/1 [00:00<00:00, 101.20it/s]

 
query_sent:  His echocardiogram was negative for cardiomyopathy.
query_word:  cardiomyopathy
Top substitutions:
1. ('fibrosis', 'The parent might not have cystic fibrosis but still might have the gene.')
2. ('pathophysiology', 'This change has occurred because the older dichotomous classification did not reflect pathophysiology or outcome.')
3. ('dystrophy', 'There is no cure for myotonic dystrophy.')
4. ('tachycardia', 'A doctor can diagnose ventricular tachycardia by doing an electrocardiogram (also called an ECG or an EKG).')
5. ('cardiogenic', 'It can also be caused by cardiogenic shock, where the heart is not getting enough blood to pump out to the rest of the body.')
6. ('arrhythmia', 'Heart   Heart disease   Cardiac arrhythmia')
7. ('Myocarditis', 'Myocarditis, also known as inflammatory cardiomyopathy, is inflammation and infection of the heart muscle.')
8. ('myocardial', 'redirect myocardial infarction')
9. ('ventricular', 'This is called idiopathic ventricular tachycardia.'



 10%|█         | 3/29 [00:21<03:11,  7.38s/it]INFO:root:Use pytorch device: cuda


Batches: 100%|██████████| 1/1 [00:00<00:00, 99.03it/s]

...

 
query_sent:  Psoriasis is generally thought to be a genetic disease that is triggered by environmental factors.
query_word:  Psoriasis
Top substitutions:
1. ('Psychosis', 'Psychosis is not a disease but rather a name for a number of symptoms, that can be caused by different diseases and conditions.')
2. ('Melanoma', 'Melanoma is prominent in New Zealand and Australia due to the hole in the ozone layer and the many beaches.')
3. ('Syphilis', 'Syphilis that is not complicated can usually be treated and cured by antibiotic medications.')
4. ('Schizophrenia', 'Schizophrenia usually appears earlier in men.')
5. ('Leprosy', 'Leprosy is an Contagious disease.')
6. ('Nocardiosis', 'Nocardiosis is a disease caused by the bacterium Nocardia Asteroides.')
7. ('Psychopathy', 'Psychopathy is a personality disorder or antisocial personality disorder.')
8. ('Neurosyphilis', 'Neurosyphilis can also appear later, usually four to 25 years after a person first got syphilis.')
9. ('Narcolepsy', 'Narcole



 97%|█████████▋| 28/29 [03:05<00:07,  7.08s/it]INFO:root:Use pytorch device: cuda


Batches: 100%|██████████| 1/1 [00:00<00:00, 100.30it/s]

 
query_sent:  There are five main types of psoriasis: plaque, guttate, inverse, pustular, and erythrodermic.
query_word:  psoriasis
Top substitutions:
1. ('leprosy', 'The symptoms of leprosy are irregular spots and patches on the skin.')
2. ('sarcoidosis', 'In some cases, pulmonary fibrosis is a complication of sarcoidosis.')
3. ('acne', 'There are lots of acne treatment methods available, including natural treatments.')
4. ('cellulite', 'The appearance of cellulite has been linked to hormonal changes, changes in diet, stress, and poor blood circulation.')
5. ('psoriatic', 'Other kinds of arthritis include psoriatic arthritis and septic arthritis (when an area is invaded by bacteria).')
6. ('Melanoma', 'Melanoma is prominent in New Zealand and Australia due to the hole in the ozone layer and the many beaches.')
7. ('rosacea', 'There are four types of rosacea, three involving human skin and the fourth affecting eyes.')
8. ('angioedema', 'In most cases, the cause of angioedema is never 



100%|██████████| 29/29 [03:12<00:00,  6.63s/it]

 
query_sent:  If one twin has psoriasis, the other twin is three times more likely to be affected if the twins are identical than if they are non-identical.
query_word:  psoriasis
Top substitutions:
1. ('schizophrenia', 'Prevention is difficult because there is no reliable way to find out in advance who will get schizophrenia.')
2. ('angioedema', 'It is possible to have angioedema without hives.')
3. ('psychosis', 'Also, if a person has psychosis that lasts longer than their mania, they may have schizoaffective disorder.')
4. ('xeroderma', 'People with xeroderma pigmentosum are about 1,000 times more likely to get skin cancer.')
5. ('acne', 'More females are getting acne than males (9.8% versus 9.0%).')
6. ('rosacea', 'Most people with rosacea have only mild redness and are never officially diagnosed or treated.')
7. ('bipolar', 'If one identical twin has bipolar disorder, there is a 40 to 70 percent chance that the other will develop bipolar disorder in their lifetime.')
8. ('psoriat

In [15]:
for query_sent, query_word in tqdm(sentences):
    output_sr_ss(query_sent, query_word, k=50)

  0%|          | 0/5 [00:00<?, ?it/s]INFO:root:Use pytorch device: cuda

 20%|██        | 1/5 [00:04<00:16,  4.13s/it]INFO:root:Use pytorch device: cuda

Batches: 100%|██████████| 1/1 [00:00<00:00, 104.21it/s]

 
query_sent:  High doses may lead to muscles contractions.
query_word:  contractions
Top substitutions:
1. ('spasms', 'The twitching is caused by the spasms.')
2. ('paralysis', 'Studies suggest that many people get sleep paralysis at least once in their lives.')
3. ('pain', 'This can cause weakness and muscle pain.')
4. ('movements', 'This medicine helps slow some of the restless movements of chorea.')
5. ('seizures', 'Coma, many seizures, or long delirium   If no treatment is given, death often occurs.')
6. ('twitches', 'Hypnic jerks are one form of involuntary muscle twitches called myoclonus.')
7. ('twitching', 'The twitching is caused by the spasms.')
8. ('muscle', 'The condition leads to general muscle stiffness and spasms in other parts of the body.')
9. ('weakness', 'This results in muscular weakness. "')
10. ('jerks', 'Benzodiazepines can make muscle jerks happen less often.')
11. ('work', 'Stress can also make the cremasteric reflex work.')
12. ('cells', 'Statins can cause da


 40%|████      | 2/5 [00:08<00:12,  4.31s/it]INFO:root:Use pytorch device: cuda

Batches: 100%|██████████| 1/1 [00:00<00:00, 104.02it/s]

 
query_sent:  High doses may lead to convulsions.
query_word:  convulsions
Top substitutions:
1. ('seizures', 'This can cause seizures.')
2. ('encephalitis', 'In 2013, encephalitis killed about 77,000 people in the world.')
3. ('hallucinations', 'Certain drugs can cause hallucinations.')
4. ('myoclonus', 'Hypnic jerks are one form of involuntary muscle twitches called myoclonus.')
5. ('vertigo', 'BPVC causes vertigo.')
6. ('dyskinesia', 'But some other people can get tardive dyskinesia too.')
7. ('neurological', 'Some forms of chorea are not linked to neurological conditions, though.')
8. ('anticonvulsant', 'For example, patients who have seizures may be given anticonvulsant medications.')
9. ('anticonvulsants', 'Other medicines that can help with psychiatric symptoms and movements are benzodiazepines and anticonvulsants usually taken for seizures.')
10. ('cerebellar', 'For example, if a person has a brain tumor in their cerebellum, they may get cerebellar ataxia.')
11. ('migraine', '


 60%|██████    | 3/5 [00:14<00:09,  4.71s/it]INFO:root:Use pytorch device: cuda

Batches: 100%|██████████| 1/1 [00:00<00:00, 39.04it/s]

 
query_sent:  To reduce the encephalocele cyst.
query_word:  encephalocele
Top substitutions:
1. ('encephalitis', 'This is called " secondary encephalitis. "')
2. ('encephalopathy', 'Transmissible spongiform encephalopathy')
3. ('encephalomyelitis', 'Neurological disorders: These are problems with the brain and the nerves, like Guillain–Barré syndrome and Post-dengue acute disseminated encephalomyelitis.')
4. ('cerebellum', 'Spinal cord injuries   Problems that damage parts of the cerebellum.')
5. ('ciliopathy', 'It is a ciliopathy.')
6. ('meningoencephalitis', 'For example:   Meningovascular syphilis, which causes seizures   General Paresis: In this brain disease, syphilis causes chronic meningoencephalitis - an infection of both the meninges and the brain which does not go away.')
7. ('ciliopathies', 'The underlying cause of the ciliopathies may be defects in the mechanism of cilia in the cell.')
8. ('prosencephalon', 'Cyclopia is characterized by a failure of the prosencephalon to 


 80%|████████  | 4/5 [00:22<00:05,  5.73s/it]INFO:root:Use pytorch device: cuda

Batches: 100%|██████████| 1/1 [00:00<00:00, 100.57it/s]

 
query_sent:  Epistaxis in children is usually from Little's area, which is on the septal wall anteriorly.
query_word:  Epistaxis
Top substitutions:
1. ('anaphylaxis', 'Epinephrine is the best treatment for anaphylaxis.')
2. ('Erythema', 'Erythema ab igne is a rash of the skin, which is usually caused by the long-term exposure to heat or infrared radiation, and which is not a burn.')
3. ('Epidemic', 'Epidemic typhus')
4. ('Anosmia', 'Anosmia is losing the the sense of smell.')
5. ('Prognathism', 'Prognathism: part of the face sticks out more than normal usually the lower jaw.')
6. ('Episodic', 'Episodic migraine (EM) is when a person has migraine symptoms for 14 days or less in one month, while chronic migraine (CM) is when a person has migraine symptoms for 15 or more days in one month.')
7. ('Narcolepsy', 'Narcolepsy affects the way the nerves work.')
8. ('Angioedema', 'Angioedema may be caused by an allergic reaction.')
9. ('Agoraphobia', 'Agoraphobia is a medical condition that ca


100%|██████████| 5/5 [00:29<00:00,  5.96s/it]

 
query_sent:  Once epistaxis occurs, the importance of the first treatment for the haemostasis should be emphasized.
query_word:  epistaxis
Top substitutions:
1. ('anaphylaxis', 'Epinephrine is the best and first treatment used for anaphylaxis.')
2. ('angioedema', 'It is possible to have angioedema without hives.')
3. ('episodic', 'Some people who start off getting episodic migraines may start to get chronic migraines later.')
4. ('phobias', 'Causes and risk factors phobias is very.')
5. ('ectoparasite', 'The mite is an ectoparasite.')
6. ('anosmia', 'In some cases, accupuncture was successfully used to treat anosmia.')
7. ('trichomoniasis', 'Treatment for trichomoniasis is usually metronidazole (Flagyl), which kills protozoa like Trichomonas vaginalis.')
8. ('Psittacosis', 'In rare cases, Psittacosis can cause endocarditis - swelling of joints, and swelling of the cornea.')
9. ('anaphylactic', 'They usually have fewer anaphylactic episodes, which are less severe, as they get older.')

In [19]:
query_sent = "He was diagnosed with cardiomyopathy in 2005 and had a pacemaker implanted."
query_word = "cardiomyopathy"
output_sr_ss(query_sent, query_word, k=50)

INFO:root:Use pytorch device: cuda
Batches: 100%|██████████| 1/1 [00:00<00:00, 97.54it/s]


 
query_sent:  He was diagnosed with cardiomyopathy in 2005 and had a pacemaker implanted.
query_word:  cardiomyopathy
Top substitutions:
1. ('arrhythmia', 'Heart   Heart disease   Cardiac arrhythmia')
2. ('ventricular', 'This is called idiopathic ventricular tachycardia.')
3. ('Myocarditis', 'Myocarditis, also known as inflammatory cardiomyopathy, is inflammation and infection of the heart muscle.')
4. ('tachycardia', 'Some people with ventricular tachycardia do not have any symptoms, especially if the V-tach lasts for only a few seconds.')
5. ('cardiogenic', 'It can also be caused by cardiogenic shock, where the heart is not getting enough blood to pump out to the rest of the body.')
6. ('dystrophy', 'There is no cure for myotonic dystrophy.')
7. ('myocardial', 'redirect myocardial infarction')
8. ('rhabdomyosarcoma', 'For example, adults with rhabdomyosarcoma usually have tumors that grow faster and are harder to treat.')
9. ('leukemia', 'Chronic leukemia grows slowly.')
10. ('brady

## Original BERT

### Ranking > selection

In [12]:
# Load data (deserialize)
with open('./simplewiki_KDTree_bertcased256.pickle', 'rb') as handle:
    tree_copy = pickle.load(handle)

In [13]:
for query_sent, query_word in tqdm(sentences):
    output_sr_ss(query_sent, query_word, k=50)

  1%|          | 1/87 [00:07<11:16,  7.87s/it]

 
query_sent:  The antibiotic we gave you in case you had meningitis can turn your tears reddish.
query_word:  meningitis
Top substitutions:
1. ('pharyngitis', 'Most of the time, it is caused by viral pharyngitis.')
2. ('meningococcal', 'Dengue can also have some of the same symptoms as other diseases, like malaria, leptospirosis, typhoid fever, and meningococcal disease.')
3. ('meningoencephalitis', 'A stiff neck is a sign that the person has either meningitis (inflammation of the meninges, which cover the brain) or meningoencephalitis (swelling of both the meninges and the brain).')
4. ('appendicitis', 'Therefore immediate treatment in hospital is needed for appendicitis.')
5. ('bronchitis', 'The medicine used to treat bronchitis may not be the same as those used to treat bronchiolitis.')
6. ('diphtheria', 'Infected people who do not know they have diphtheria are called carriers of diphtheria, because they can spread the infection without being sick themselves.')
7. ('meningioma', 'a

  2%|▏         | 2/87 [00:16<11:33,  8.15s/it]

 
query_sent:  They swap spit, virus travels to his brain, leads to meningitis, leads to increased intracranial pressure.
query_word:  meningitis
Top substitutions:
1. ('conjunctivitis', 'People who do have symptoms usually have a low fever, conjunctivitis, joint pain (mainly in the hands and feet), and a rash.')
2. ('bronchitis', 'They can be bronchitis or pneumonia.')
3. ('pharyngitis', 'Secondary bacterial infections may occur resulting in sinusitis, pharyngitis, or an ear infection.')
4. ('sinusitis', 'Brain tumor   Lyme disease   Influenza   Brain metastasis   Foodborne Intoxication   Sinusitis   Meningitis   Encephalitis   In many cases headaches can be relieved naturally.')
5. ('meningococcal', 'Dengue can also have some of the same symptoms as other diseases, like malaria, leptospirosis, typhoid fever, and meningococcal disease.')
6. ('hypoglycemia', 'Causes of pallor may include migraines, headache, hypoglycemia, anemia or scarlet fever.')
7. ('pancreatitis', 'Causes include p

  3%|▎         | 3/87 [00:23<10:55,  7.81s/it]

...

 
query_sent:  To reduce the encephalocele cyst.
query_word:  encephalocele
Top substitutions:
1. ('rhabdomyosarcoma', 'This is the most common type of rhabdomyosarcoma.')
2. ('encephalopathy', 'Transmissible spongiform encephalopathy')
3. ('cystocele', 'The doctor who fixes the cystocele may fold the weak tissue and then stitch it back in place.')
4. ('toxoplasmosis', 'Toxoplasmosis Toxoplasma gondii parasite Pigs, lambs, deer, cattle Eating under-cooked meat.')
5. ('bronchiectasis', 'René Laennec wrote the first description of bronchiectasis in 1819.')
6. ('hepatosplenomegaly', 'Hepatomegaly -enlarged liver, splenomegaly - enlarged spleen, or both hepatosplenomegaly.')
7. ('ovary', 'It is a pus-filled ovary and Fallopian tube that is filled with infection.')
8. ('astrocytomas', 'Low-grade astrocytomas usually grow slowly and stay in one place.')
9. ('epithelium', 'RSV does cause epithelium damage.')
10. ('toxoplasma', 'Toxoplasmosis Toxoplasma gondii parasite Cats Cat feces (in cat l

 99%|█████████▉| 86/87 [09:41<00:06,  6.44s/it]

 
query_sent:  Epistaxis in children is usually from Little's area, which is on the septal wall anteriorly.
query_word:  Epistaxis
Top substitutions:
1. ('anaphylaxis', 'Anaphylaxis appears to be getting more common.')
2. ('symptom', 'This symptom may be harder to see in infants who cannot walk yet.')
3. ('epiglottitis', 'Epiglottitis usually happens in children.')
4. ('hypotension', 'Hypotension is low blood pressure.')
5. ('hypoglycemia', 'Hypoglycemia is treated by returning the blood sugar levels back to normal.')
6. ('cyanosis', 'Usually cyanosis is first seen in the extremities (the parts of the body that are farthest away from the heart).')
7. ('ischemia', 'Mesenteric ischemia happens mostly to the elderly (older people).')
8. ('sarcoidosis', 'Sarcoidosis may involve the brain.')
9. ('nystagmus', 'Nystagmus can be hereditary, but it can also be a result of something else.')
10. ('vasculitis', 'Vasculitis is primarily caused by leukocyte migration and resultant damage.')
11. ('cy

100%|██████████| 87/87 [09:48<00:00,  6.77s/it]

 
query_sent:  Once epistaxis occurs, the importance of the first treatment for the haemostasis should be emphasized.
query_word:  epistaxis
Top substitutions:
1. ('anaphylaxis', 'Anaphylaxis usually happens because the immune system over-reacts to an allergen.')
2. ('hypoglycemia', 'Hypoglycemia is treated by returning the blood sugar levels back to normal.')
3. ('epilepsy', 'Most forms of epilepsy cannot be cured.')
4. ('encephalitis', 'Encephalitis can cause serious symptoms, like seizures and strokes, and can be fatal.')
5. ('prosopagnosia', 'Prosopagnosia is not curable or treatable.')
6. ('trichomoniasis', 'About 95% to 97% of people with trichomoniasis are cured after one dose of metronidazole.')
7. ('neurosyphilis', 'Instead, people with neurosyphilis usually need to be given large doses of penicillin for at least 10 days.')
8. ('prostatitis', 'Sepsis from prostatitis is very rare, but may occur in immunocompromised patients; high fever and malaise generally prompt blood cultur